# Introduction

This notebook is intended to explain various APIs available in PyDough to explain and explore the PyDough metadata and PyDough logical operations.

## Setup

This notebook uses the same setup configuration as the `Introduction` notebook, which much be executed so that PyDough is initialized with the desired data.

In [1]:
import pydough

pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH")
%load_ext pydough_jupyter_extensions

## Using `pydough.explain` on metadata

TODO

In [2]:
graph = pydough.active_session.metadata

pydough.explain(graph)

AttributeError: module 'pydough' has no attribute 'explain'

TODO